# Ceph OSD json file format

In [2]:
import json
import glob as glob

## Construction 1. A Simple Hierachical Parser

In [1]:
from copy import deepcopy

def indexableNodes(nodes):
    return {str(node["id"]):node for node in nodes}

def getRoots(nodes):
    ROOT_TYPE = "root" # ROOT_ID = 10
    return [ k for k in nodes if nodes[k]["type"] == ROOT_TYPE]


In [20]:

def getTree(nodes):
    roots = getRoots(nodes);
    return {
        "name": "_root",
        "children": [getTree_(i, nodes) for i in roots]
    };


def getTree_(i, nodes):
    """
        i: string-typed index
        nodes: the indexable nodes representation
    """
    node = nodes[i]
    
#     print(i, type(i), node)

    if node["type"] == "osd":
        status = node["status"]
        ret = { 
            "name": node["name"], 
            "status": status, # Temporary indication of the 
            "data": node
        }
        return ret;
    
    children = node["children"]
    cnode = [getTree_(str(n), nodes) for n in children]
    ret = {
        "name": node["name"],
        "children": cnodes,
        "data": node
    }
    return ret

## Test

In [129]:
cd data

[Errno 2] No such file or directory: 'data'
/Users/mike/Documents/GitHub/cs739-osdvisual/data


In [131]:
data = json.load(open("rawdata/beesly.json"))
nodes = indexableNodes(data["nodes"])
t = getTree(nodes)
with open("processed.json", "w+") as f:
    json.dump(t, f)

'getTree'  6.39 ms


## Production

- Source json directory: `rawdata`
- Product json directory: `product`

In [21]:
import os

rawdata_dir = "rawdata"
product_dir = "product"

try:
    os.mkdir(product_dir)
    print("Creat product directory")
except:
    print("Exist product directory")


# Find all json files in the rawdata directory
jsonFiles = glob.glob("{0}/*.json".format(rawdata_dir))
for fs in jsonFiles:
    # Construct the path to store data
    basename = os.path.basename(fs)
    fdname = os.path.splitext(basename)[0]
    fdpath = product_dir + "/" + fdname + ".product.json"    
    with open(fs) as f:
        data = json.load(f)
        t = getTree(indexableNodes(data["nodes"]))  
        with open(fdpath, "w+") as fd:
            print("Write {0}".format(fdpath))
            json.dump(t, fd)

Exist product directory
Write product/jim.product.json
Write product/erin.product.json
Write product/kelly.product.json
Write product/beesly.product.json


## Minimal Production
- Internal node: preserve a path
- Leaf: perserve the status (as size)

In [18]:
def getminiTree(nodes):
    roots = getRoots(nodes);
    return {
        "name": "_root", # Recursion below
        "children": [getminiTree_(i, nodes) for i in roots]
    };


def getminiTree_(i, nodes):
    """
        i: string-typed index
        nodes: the indexable nodes representation
    """
    node = nodes[i]
    
    if node["type"] == "osd":
        status = 1 if node["status"] == "up" else 0
        ret = { 
            "name": node["name"], 
            "size": status,
            "type": node["type"]
        }
        return ret;
    
    children = node["children"]
    ret = {
        "name": node["name"], 
        "type": node["type"], # Recursion below
        "children": [getminiTree_(str(n), nodes) for n in children],

    }
    return ret

In [19]:
import os
import json
import glob as glob

rawdata_dir = "rawdata"
product_dir = "miniproduct"

try:
    os.mkdir(product_dir)
    print("Creat product directory")
except:
    print("Exist product directory")


# Find all json files in the rawdata directory
jsonFiles = glob.glob("{0}/*.json".format(rawdata_dir))
for fs in jsonFiles:
    # Construct the path to store data
    basename = os.path.basename(fs)
    fdname = os.path.splitext(basename)[0]
    fdpath = product_dir + "/" + fdname + ".product.json"
    with open(fs) as f:
        data = json.load(f)
        t = getminiTree(indexableNodes(data["nodes"]))
        with open(fdpath, "w+") as fd:
            print("Write {0}".format(fdpath))
            json.dump(t, fd)

Exist product directory
Write miniproduct/jim.product.json
Write miniproduct/erin.product.json
Write miniproduct/kelly.product.json
Write miniproduct/beesly.product.json


## Legacy Code

### Basics Properties of the json files
1. **Multiple `roots` allowed**. Different roots might belong to different cluster (`default` cluster, `drain` cluster -- shown in `beesly.json`). `"stray"` nodes should also be taken care of.
2. **Hierachical structure varied**. It is much better to handle the hierachical structure as directory hierachy. We already seen hierachical structure as follows:
    - `root -> host`
    - `root -> host -> osd`
    - `root -> rack -> host -> osd`

In [4]:
cd data

[Errno 2] No such file or directory: 'data'
/Users/mike/Documents/GitHub/cs739-osdvisual/data


In [5]:
jsonFiles = glob.glob("product/*.json")
dataPool = {} # Read Json Data into Pool
for fs in jsonFiles:
    with open(fs) as f:
        dataPool[fs] = json.load(f)

In [6]:
print(dataPool.keys())

dict_keys(['product/beesly.json.product.json', 'product/jim.product.json', 'product/kelly.json.product.json', 'product/erin.product.json', 'product/kelly.product.json', 'product/erin.json.product.json', 'product/beesly.product.json', 'product/jim.json.product.json'])


In [7]:
def analyzeJsonData(jsonData):
    # 1. Multiple roots are possible
    nodes = data["nodes"]
    idset = set(map(lambda x: x["id"], nodes))
    childrenset = reduce(lambda x,y: set(x) | set(y), 
                         map(lambda x: x.get("children", []), nodes))
    print("    - root(s): {0}".format(childrenset ^ idset))
    
    # 2. OSD have no children, other should have children
    
    # 3. Maximum number of children varies (in this case, 48)
    children_length = map(lambda x: len(x.get("children", [])), nodes)
    print("    - max children in a node: {0}".format(max(children_length)))
    return 

    
for k in dataPool:
    print("-- {} --".format(k))
    data = dataPool[k]
    analyzeJsonData(data)

-- product/beesly.json.product.json --


KeyError: 'nodes'